In [ ]:
import os
import random
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
for dirpath, dirname, filenames in os.walk("../input/far-new/Dataset"):
    print(f"There are {len(dirname)} directories and there are {len(filenames)} images in {dirpath}")

In [ ]:
#exp = random.choice(["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"])
exp = input("Enter the class : ")
data_set = random.choice(["train", "test"])

rows, cols = 4, 10
plt.figure(figsize = (20, 10))

target_dir = f"../input/far-new/Dataset/Dataset/{data_set}/{exp}" 
random_images = random.sample(os.listdir(target_dir), 40)

for row in range(rows):
    for col in range(cols):
        index = row * cols + col
        plt.subplot(rows, cols, index + 1)
        plt.title(exp)
        img = mpimg.imread(f"{target_dir}/{random_images[index]}")
        plt.imshow(img)
        plt.axis("off");

In [ ]:
train_dir = "../input/far-new/Dataset/Dataset/train"
test_dir = "../input/far-new/Dataset/Dataset/test"

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                image_size = (128, 128),
                                                                shuffle = True,
                                                                label_mode = "categorical")

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                               image_size = (128, 128),
                                                               shuffle = True,
                                                               label_mode = "categorical")
train_data, test_data

In [ ]:
train_dataset = train_data.prefetch(tf.data.AUTOTUNE)
test_dataset = test_data.prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

In [ ]:
data_augmentation_layer = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomWidth(0.1),
    preprocessing.RandomHeight(0.1)
])

In [ ]:
# Setup checkpoint path
checkpoint_path = "90_83/checkpoint.ckpt" # note: remember saving directly to Colab is temporary

# Create a ModelCheckpoint callback that saves the model's weights only
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         monitor = 'val_accuracy',
                                                         save_best_only = True,
                                                         verbose=1)

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
     data_augmentation_layer,
     layers.Conv2D(64, 3, activation = "relu", input_shape = (128, 128, 3)),
     layers.Conv2D(64, 3, activation = "relu"),
     layers.MaxPooling2D(2),
     layers.Conv2D(128, 3, activation = "relu"),
     layers.Conv2D(128, 3, activation = "relu"),
     layers.MaxPooling2D(2),
     layers.Conv2D(256, 3, activation = "relu"),
     layers.Dropout(0.25),
     layers.Conv2D(256, 3, activation = "relu"),
     layers.Dropout(0.25),
     layers.MaxPooling2D(2),
     layers.Conv2D(512, 3, activation = "relu"),
     layers.Dropout(0.5),
     layers.GlobalAveragePooling2D(),
     layers.Dense(512, activation = "relu"),
     layers.Dropout(0.25),
     layers.Dense(512, activation = "relu"),
     tf.keras.layers.Dense(7, activation = "softmax")
])

model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(0.0006), metrics = ["accuracy"])


model_history = model.fit(train_dataset,
                         epochs = 30,
                         validation_data = test_dataset,
                         callbacks = [checkpoint_callback])

In [ ]:
pd.DataFrame(model_history.history).plot()

In [ ]:
model.save("Final_tweak")

In [ ]:
model.save("Final_tweak.h5")

In [ ]:
import shutil
shutil.make_archive('Final_tweak', 'zip', './Final_tweak')